In [7]:
import getpass
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
import requests
import os

print('Modules Loaded')

Modules Loaded


In [8]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    # prompt the user to enter their AGOL username
    username = input("Enter your ArcGIS Online username:")
    
    # prompt the user to enter their AGOL password (without echoing)
    password = getpass.getpass("Enter your ArcGIS Onine password:")
    
    # create a GIS object and login to AGOL
    gis = GIS("https://www.arcgis.com", username, password)
    
    # return the object
    return gis

# run module
gis = login_AGOL()

# confirm user login 
print("Logged in as:", gis.users.me.username)

Enter your ArcGIS Online username:Stacey_Vernon_GES23
Enter your ArcGIS Onine password:········
Logged in as: Stacey_Vernon_GES23


In [9]:
# Utilise the AGOL API and the a "static" GeoJSON file to publish data to a feature service. Specifically publish
# Dublin Bike zone data.
# documentation on using the ArcGIS API to create a feature service from GeoJSON here: https://developers.arcgis.com/python/guide/import-data/
# We will determine if a feature layer exists, if not we will create one but if it does - we skip this function

# generate a list of the below search conditions in our AGOL Item Content
zoneSearch = gis.content.search("title:BikeStations_Zones",item_type="Feature Layer")

# if list is empty, create a new feature layer
if not zoneSearch:

# set up the feature service properties in AGOL for the stations and zone dataset
    item_properties = {
    "title": "BikeStations_Zones", #provide a title for your item
    "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }

    # URL link to the GeoJSON file
    geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
    geojson_item = gis.content.add(item_properties, geojson_url)

    # publish the GeoJSON as a feature layer
    BikeStationZone_data = geojson_item.publish()
    print("Data published. New feature layer created.")
else:
    print("Bike Station and Zone data already exists in AGOL.")

Bike Station and Zone data already exists in AGOL.


In [10]:
# Dublin Bike locations are available as a live dataset, updated every 5 minutes
# pull in the Dublin Bike locations from the API

# URL to API endpoint for bike locations
bike_url = "https://data.smartdublin.ie/bleeperbike-api/bikes/bleeper_bikes/current/bikes.geojson"

status = False
while status != True: 
    response = requests.get(bike_url)
    # if the request was successful (status code 200), get the data
    if response.status_code in [200,201]:
        # parse the JSON response
        bikeData = response.json()
        status = True
        print ('Successfully retrieved data')
    else:
        status = False
        time.sleep(60)
        print ('Failed to retrieve data, trying again.')    

Success


In [14]:
# create a FLC from the GeoJSON

# set up the feature service properties in AGOL for the stations and zone dataset
item_properties = {
    "title": "Bike_Locations", #provide a title for your item
    "description": "Dublin Bike current locations", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJson" # define the type of data you are trying to pubish
    }
    
# search AGOL contents to check if the layer already exists
fl_search = gis.content.search("title:Bike_Locations",item_type="Feature Layer")

if not fl_search:
    # if search list is empty (meaning a feature layer with that title does not exist), upload the GeoJSON 
    # & publish as a new feature service
    geojson_item = gis.content.add(item_properties, bike_url)
    live_bike_fl = geojson_item.publish()
    print("Successfully created new feature service and published live bike location")
    
else: 
    # if search list is NOT empty (meaning feature layer with that title already exists), overwrite the data within the FL
    live_data_fl = fl_search[0]
    content = FeatureLayerCollection.fromitem(live_data_fl)
    overwrite_result = content.manager.overwrite(bike_url)
    
    # overwrite_result will have {"success":True} or {"success":False}
    if not overwrite_result['success']:
        # if not true
        print("Failed to overwrite Feature Layer Collection with new GeoJSON Data")
    else:
        # if true
        print("Successfully updated Feature Layer Collection with new GeoJSON Data")

Successfully created new feature service and published live bike location


In [ ]:
# need to do:
# improve commenting 
# add in docstrings
# publish live bike data
# get live bike data to refresh
# display in webmap
# product heatmap of current locations - deduce busy areas??